In [1]:
#%load_ext autoreload
#%autoreload 2

#import sys
#sys.path.insert(0,'..')

import os

import backtrader as bt
import backtrader.indicators as btind
import numpy as np
import scipy.signal as signal
from scipy import stats

from gym import spaces

from btgym import BTgymEnv, BTgymStrategy, BTgymDataset

from launcher import Launcher

In [ ]:
# GYM TEST ENV:
cluster_config = dict(
    host='127.0.0.1',
    port=12222,
    num_workers=1,
    num_ps=1,
    log_dir='./tmp/a3c_testing_gym',
)

env_config = dict(
    gym_id='Breakout-v0'
)


launcher = Launcher(
    cluster_config=cluster_config,
    env_config=env_config,
    train_steps=500,
    opt_learn_rate=1e-4,
    rollout_length=20,
    test_mode=True,
    model_summary_freq=50,
    episode_summary_freq=2,
    env_render_freq=10,
    verbose=2
)

In [ ]:
launcher.run()


In [13]:
class MyStrategy(BTgymStrategy):
    """
    Example subclass of BT server inner computation startegy.
    """
    
    def __init__(self, **kwargs):
        super(MyStrategy,self).__init__(**kwargs)
        
        self.order_penalty = 0.5
        self.trade_just_closed = False
        self.trade_result = None
        
    def notify_trade(self, trade):
        #if trade.justopened:
            #print('trade {} just opened'.format(trade.ref))
            
        if trade.isclosed:
            #print('trade {} closed, pnl_comm: {}'.format(trade.ref, trade.pnlcomm))
            # Set trade flag and result:
            self.trade_just_closed = True
            self.trade_result = trade.pnlcomm
    
    def sigmoid(self, x):
        return 1/(1 + np.exp(-x))
        
    def get_state(self):
        """
        Computes featurized RL-ready environment observation state
        by applying continious wavelet transform to time-embedded vector
        of close-price gradients.
        """
        # Use close price:
        channel = 3
        
        X = self.raw_state[:, channel]
        
        # Prepare parameters:
        Tau = 5
        max_cwt_scale = self.p.state_shape['model_input'].shape[1]
        cwt_width = np.linspace(Tau, max_cwt_scale + Tau - 1, max_cwt_scale) # scale of wavelet transdorm [n]
    
        T = 1# 000
        
        # Get vector of gradients of last [n] prices:
        X = np.gradient(X, axis=0) * T
        
        # Compute continious wavelet transform using Ricker wavelet, get [n,m,1]-dim. matrix:
        X = signal.cwt(X, signal.ricker, cwt_width).T[:, :, None]
        
        # Z-score:
        X = stats.zscore(X)
        
        # Local min-max norm:
        self.state['model_input'] = (X - X.min()) / (X.max() - X.min())
        
        # Squash values in [0,1]:
        #self.state['model_input'] = self.sigmoid(self.state['model_input'])
        
        return self.state

    
    def get_reward(self):
        """
        Defines reward as [0,1]-bounded function of last closed trade result.
        """
        r = 0
        
        # Result
        if self.trade_just_closed:
            r = self.trade_result
            self.trade_just_closed = False
            
        # Penalty for failed order:
        if self.order_failed:
            #print('Failed order!')
            r -= self.order_penalty
            self.order_failed = False
            
        #print('reward_', r)
        
        return r / 20
            
# Set backtesting engine parameters:

state_shape = {
    'raw_state': spaces.Box(low=-1, high=1, shape=(120, 4)),
    'model_input': spaces.Box(low=-10, high=10, shape=(120, 15, 1))
}

MyCerebro = bt.Cerebro()

MyCerebro.addstrategy(
    MyStrategy,
    state_shape=state_shape,
    portfolio_actions=('hold', 'buy', 'sell'),
    drawdown_call=5, # in percent of initial cash
    target_call=50,
    skip_frame=10,
)

# Set leveraged account:
MyCerebro.broker.setcash(2000)
MyCerebro.broker.setcommission(commission=0.0001, leverage=10.0)
MyCerebro.broker.set_shortcash(False)
MyCerebro.addsizer(bt.sizers.SizerFix, stake=10000,)


MyCerebro.addanalyzer(bt.analyzers.DrawDown)

# Provide data (seven years of 1 minute bars):
filenames = [
    '../data/DAT_ASCII_EURUSD_M1_2010.csv',
    '../data/DAT_ASCII_EURUSD_M1_2011.csv',
    '../data/DAT_ASCII_EURUSD_M1_2012.csv',
    '../data/DAT_ASCII_EURUSD_M1_2013.csv',
    '../data/DAT_ASCII_EURUSD_M1_2014.csv',
    '../data/DAT_ASCII_EURUSD_M1_2015.csv',
    '../data/DAT_ASCII_EURUSD_M1_2016.csv',
]

MyDataset = BTgymDataset(
    filename=filenames,
    #filename='../data/test_sine_1min_period256_delta0002.csv',
    start_weekdays=[0, 1, 2, 3, 4],
    episode_len_days=0,
    episode_len_hours=23,
    episode_len_minutes=55,
    start_00=True,
    time_gap_hours=6,
)
env_config = dict(
    dataset=MyDataset,
    engine=MyCerebro,
    render_modes=['episode', 'human', 'model_input'],
    render_state_as_image=True,
    render_ylabel='Z-norm/CWT',
    render_size_episode=(12,8),
    render_size_human=(8, 3.5),
    render_size_state=(10, 5),
    render_dpi=75,
    port=5000,
    data_port=4999,
    connect_timeout=60,
    verbose=0,
)
cluster_config = dict(
    host='127.0.0.1',
    port=12222,
    num_workers=8,
    num_ps=1,
    log_dir='./tmp/a3c_testing',
)
launcher = Launcher(
    cluster_config=cluster_config,
    env_class=BTgymEnv,
    env_config=env_config,
    rollout_length=20,
    test_mode=False,
    train_steps=1000000000,
    model_summary_freq=20,
    episode_summary_freq=1,
    env_render_freq=20,
    verbose=1
    
)

In [14]:
launcher.run()

[2017-08-24 09:28:50,145] Press `Ctrl-C` to stop training and close launcher.


Press `Ctrl-C` to stop training and close launcher.
INFO:tensorflow:Restoring parameters from ./tmp/a3c_testing/train/model.ckpt-14231752


[2017-08-24 09:28:57,697] Restoring parameters from ./tmp/a3c_testing/train/model.ckpt-14231752


INFO:tensorflow:Waiting for model to be ready.  Ready_for_local_init_op:  None, ready: Variables not initialized: global/l1/W, global/l1/b, global/l2/W, global/l2/b, global/l3/W, global/l3/b, global/l4/W, global/l4/b, global/rnn/basic_lstm_cell/kernel, global/rnn/basic_lstm_cell/bias, global/action/w, global/action/b, global/value/w, global/value/b, global/global_step, global/global_episode, beta1_power, beta2_power, global/l1/W/Adam, global/l1/W/Adam_1, global/l1/b/Adam, global/l1/b/Adam_1, global/l2/W/Adam, global/l2/W/Adam_1, global/l2/b/Adam, global/l2/b/Adam_1, global/l3/W/Adam, global/l3/W/Adam_1, global/l3/b/Adam, global/l3/b/Adam_1, global/l4/W/Adam, global/l4/W/Adam_1, global/l4/b/Adam, global/l4/b/Adam_1, global/rnn/basic_lstm_cell/kernel/Adam, global/rnn/basic_lstm_cell/kernel/Adam_1, global/rnn/basic_lstm_cell/bias/Adam, global/rnn/basic_lstm_cell/bias/Adam_1, global/action/w/Adam, global/action/w/Adam_1, global/action/b/Adam, global/action/b/Adam_1, global/value/w/Adam, gl

[2017-08-24 09:28:59,001] Waiting for model to be ready.  Ready_for_local_init_op:  None, ready: Variables not initialized: global/l1/W, global/l1/b, global/l2/W, global/l2/b, global/l3/W, global/l3/b, global/l4/W, global/l4/b, global/rnn/basic_lstm_cell/kernel, global/rnn/basic_lstm_cell/bias, global/action/w, global/action/b, global/value/w, global/value/b, global/global_step, global/global_episode, beta1_power, beta2_power, global/l1/W/Adam, global/l1/W/Adam_1, global/l1/b/Adam, global/l1/b/Adam_1, global/l2/W/Adam, global/l2/W/Adam_1, global/l2/b/Adam, global/l2/b/Adam_1, global/l3/W/Adam, global/l3/W/Adam_1, global/l3/b/Adam, global/l3/b/Adam_1, global/l4/W/Adam, global/l4/W/Adam_1, global/l4/b/Adam, global/l4/b/Adam_1, global/rnn/basic_lstm_cell/kernel/Adam, global/rnn/basic_lstm_cell/kernel/Adam_1, global/rnn/basic_lstm_cell/bias/Adam, global/rnn/basic_lstm_cell/bias/Adam_1, global/action/w/Adam, global/action/w/Adam_1, global/action/b/Adam, global/action/b/Adam_1, global/value/

...

INFO:tensorflow:Saving checkpoint to path ./tmp/a3c_testing/train/model.ckpt


[2017-08-24 12:29:02,105] Saving checkpoint to path ./tmp/a3c_testing/train/model.ckpt


INFO:tensorflow:Saving checkpoint to path ./tmp/a3c_testing/train/model.ckpt


[2017-08-24 12:34:02,101] Saving checkpoint to path ./tmp/a3c_testing/train/model.ckpt
Process BTgymServer-47:1:
[2017-08-24 12:34:13,530] worker_1 has joined.
Process BTgymServer-44:1:
[2017-08-24 12:34:13,537] worker_2 has joined.
Process BTgymServer-43:1:
Process BTgymServer-42:1:
Process BTgymServer-40:2:
Process BTgymServer-45:1:
Process BTgymServer-46:1:
Process BTgymServer-41:1:
Process BTgymDataFeedServer-40:1:
[2017-08-24 12:34:13,577] worker_3 has joined.
[2017-08-24 12:34:13,602] worker_4 has joined.
[2017-08-24 12:34:13,603] worker_5 has joined.
[2017-08-24 12:34:13,603] worker_6 has joined.
[2017-08-24 12:34:13,604] worker_7 has joined.
[2017-08-24 12:34:13,605] chief_worker_0 has joined.
[2017-08-24 12:34:13,606] parameter_server_0 has joined.
[2017-08-24 12:34:13,606] Launcher closed.
Traceback (most recent call last):
  File "/Users/muzikin/anaconda/envs/tensorforce/lib/python3.6/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/Users/muzikin/

  File "/Users/muzikin/anaconda/envs/tensorforce/lib/python3.6/site-packages/backtrader/analyzer.py", line 188, in _next
    self.next()
  File "/Users/muzikin/Yandex.Disk.localized/work/btgym/btgym/server.py", line 103, in next
    state = self.strategy.get_state()
  File "/Users/muzikin/Yandex.Disk.localized/work/btgym/btgym/server.py", line 109, in next
    self.message = self.socket.recv_pyobj()
  File "/Users/muzikin/anaconda/envs/tensorforce/lib/python3.6/site-packages/zmq/sugar/socket.py", line 491, in recv_pyobj
    msg = self.recv(flags)
  File "<ipython-input-13-f51f9f3b0961>", line 45, in get_state
    X = np.gradient(X, axis=0) * T
  File "zmq/backend/cython/socket.pyx", line 693, in zmq.backend.cython.socket.Socket.recv (zmq/backend/cython/socket.c:7683)
  File "/Users/muzikin/anaconda/envs/tensorforce/lib/python3.6/site-packages/numpy/lib/function_base.py", line 1763, in gradient
    out[slice1] = (f[slice4] - f[slice2]) / (2. * dx[i])
  File "zmq/backend/cython/socket.py

In [ ]:
print(launcher.kwargs, '\n\n')
print(launcher.env_config)
print(launcher.cluster_config)
print(launcher.cluster_spec)
for config in launcher.workers_config_list:
    print('============')
    for k, v in config.items():
        print('{}:\n{}\n'.format(k, v))
    

In [ ]:
def func1(max_step):
    step = 0
    done = False
    
    def func2(max_step):
        nonlocal step
        nonlocal done
        step +=1
        if step == max_step:
            step = 0
            done = True
        return step
    
    for i in range(20):
        done = False
        print(func2(max_step), step, done)
        


func1(7)
            

In [ ]:
a = dict()
a.update({'b': 2, 'c':4})
type(a) == dict